In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pdF
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/ssd/ssd300_coco.py')

root='../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/ssd300_coco_trash'

#cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

AttributeError: 'ConfigDict' object has no attribute 'roi_head'

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/ssd300_coco_trash/latest.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: bbox_head.cls_convs.0.0.weight, bbox_head.cls_convs.0.0.bias, bbox_head.cls_convs.1.0.weight, bbox_head.cls_convs.1.0.bias, bbox_head.cls_convs.2.0.weight, bbox_head.cls_convs.2.0.bias, bbox_head.cls_convs.3.0.weight, bbox_head.cls_convs.3.0.bias, bbox_head.cls_convs.4.0.weight, bbox_head.cls_convs.4.0.bias, bbox_head.cls_convs.5.0.weight, bbox_head.cls_convs.5.0.bias, bbox_head.reg_convs.0.0.weight, bbox_head.reg_convs.0.0.bias, bbox_head.reg_convs.1.0.weight, bbox_head.reg_convs.1.0.bias, bbox_head.reg_convs.2.0.weight, bbox_head.reg_convs.2.0.bias, bbox_head.reg_convs.3.0.weight, bbox_head.reg_convs.3.0.bias, bbox_head.reg_convs.4.0.weight, bbox_head.reg_convs.4.0.bias, bbox_head.reg_convs.5.0.weight, bbox_head.reg_convs.5.0.bias, backbone.features.0.weight, backbone.features.0.bias, backbone.features.2.weight, backbone.

In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 25.6 task/s, elapsed: 190s, ETA:     0s

In [7]:
import pandas as pd

# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,2 0.10357691 416.36646 655.38684 484.30603 705...,test/0000.jpg
1,2 0.10059084 163.49829 875.8898 229.06757 934....,test/0001.jpg
2,2 0.10438022 0.0 281.1042 30.652523 340.7962 2...,test/0002.jpg
3,2 0.10523136 618.8773 60.064167 722.15247 108....,test/0003.jpg
4,2 0.10085052 569.7789 611.93506 634.0485 666.7...,test/0004.jpg
